In [46]:
import io
import json
import pandas
import requests
from typing import TypedDict

In [47]:
letters = ["a", "b", "c", "d", "e"]
url = "https://raw.githubusercontent.com/lichess-org/chess-openings/master"

openings_data_frame = pandas.DataFrame()

for letter in letters:
    print("letter", letter)
    letter_url = f"{url}/{letter}.tsv"
    response = requests.get(letter_url)
    tsv = response.text
    df = pandas.read_csv(io.StringIO(tsv), delimiter="\t")
    openings_data_frame = pandas.concat([openings_data_frame, df])

letter a
letter b
letter c
letter d
letter e


In [48]:
class Opening(TypedDict):
    eco: str
    name: str
    pgn: str

In [49]:
openings_data_frame.to_csv("./all/openings.csv", index=False)
openings_string_json: str = openings_data_frame.to_json(orient="records")

In [50]:
from io import TextIOWrapper

openings: str = json.loads(openings_string_json)
openings_string_json_file: TextIOWrapper = open("./all/openings.json", "w", encoding="utf-8")
json.dump(openings_string_json, openings_string_json_file, ensure_ascii=False, indent=2)

In [51]:

lessons_openings = []

for opening in openings:
    pgn : str = opening.get("pgn")
    if "10." in pgn:
        lessons_openings.append(opening)

In [52]:
lessons_openings_string_json_file = open("./lessons/openings.json", "w", encoding="utf-8")
json.dump(lessons_openings, lessons_openings_string_json_file, ensure_ascii=False, indent=2)

In [53]:
def get_moves(pgn: str) -> list[str]:
    items: list[str] = pgn.split(" ")
    moves: list[str] = [item for index, item in enumerate(items) if index % 3 != 0]
    return moves

In [54]:
def get_half_moves(pgn: str) -> int:
    moves = get_moves(pgn)
    return len(moves)

In [55]:
def get_main_line_score(pgn: str, pgn_list: list[str]):
    moves_list: list[list[str]] = [get_moves(pgn_item) for pgn_item in pgn_list]
    moves : list[str] = get_moves(pgn)
    index : int = 1
    flag = True
    while flag:
        moves_string : str = "-".join(moves[0:index])
        moves_contained = [
            "-".join(moves_item)
            for moves_item in moves_list
            if "-".join(moves_item).startswith(moves_string)
        ]
        if len(moves_contained) <= 1 or len(moves) == index:
            flag = False
        else:
            index += 1
    print(pgn, index)
    return index

In [56]:
import chess.pgn

def get_fen(pgn: str) -> str:
    pgn_string = io.StringIO(pgn)
    game = chess.pgn.read_game(pgn_string)
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board.fen()

In [57]:
from stockfish import Stockfish

stockfish = Stockfish()

def get_evaluation(fen: str) -> float:
    stockfish.set_fen_position(fen)
    evaluation : dict = stockfish.get_evaluation()
    centipawn = evaluation.get("value", 0)
    pawn = centipawn / 100
    return pawn

In [58]:
lessons_openings_data_frame = pandas.DataFrame(lessons_openings)
pgn_list: list[str] = lessons_openings_data_frame['pgn'].tolist()
lessons_openings_data_frame["first"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: f"{pgn.split(" ")[1]}-{pgn.split(" ")[2]}")
lessons_openings_data_frame["half_moves"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_half_moves(pgn))
lessons_openings_data_frame["fen"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_fen(pgn))
lessons_openings_data_frame["evaluation"] = lessons_openings_data_frame["fen"].apply(lambda fen: get_evaluation(fen))
lessons_openings_data_frame["advantage"] = lessons_openings_data_frame["evaluation"].apply(lambda evaluation: "white" if evaluation > 0 else "black")
lessons_openings_data_frame["main_line_score"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_main_line_score(pgn, pgn_list))
lessons_openings_data_frame = lessons_openings_data_frame.rename(columns={"name": "opening_variation"})
lessons_openings_data_frame["opening"] = lessons_openings_data_frame["opening_variation"].apply(lambda variation: f"{variation.split(":")[0]}")
lessons_openings_data_frame = lessons_openings_data_frame.sort_values(by=['first', 'opening', 'pgn', 'main_line_score'])
lessons_openings_data_frame.to_csv("./lessons/openings.csv", index=False)

1. c4 e6 2. Nf3 c5 3. Nc3 Nf6 4. g3 b6 5. Bg2 Bb7 6. O-O Be7 7. d4 cxd4 8. Qxd4 d6 9. Rd1 a6 10. b3 Nbd7 1
1. d4 Nf6 2. c4 c5 3. d5 b5 4. cxb5 a6 5. bxa6 g6 6. Nc3 Bxa6 7. Nf3 d6 8. e4 Bxf1 9. Kxf1 Bg7 10. g3 O-O 11. Kg2 6
1. d4 Nf6 2. c4 e6 3. g3 c5 4. d5 exd5 5. cxd5 d6 6. Nc3 g6 7. Bg2 Bg7 8. Nf3 O-O 9. O-O a6 10. a4 Nbd7 11. Nd2 Re8 6
1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O a6 10. a4 18
1. d4 Nf6 2. c4 e6 3. Nf3 c5 4. d5 exd5 5. cxd5 d6 6. Nc3 g6 7. e4 Bg7 8. Be2 O-O 9. O-O a6 10. a4 Bg4 18
1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2 19
1. d4 Nf6 2. c4 e6 3. Nf3 c5 4. d5 exd5 5. cxd5 d6 6. Nc3 g6 7. e4 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2 Na6 18
1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2 Na6 11. f3 20
1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. c4 Nb6 5. exd6 cxd6 6. Nc3 g6 7. h3 Bg7 8. Nf3 O-O 9. Be2 Nc6 10. O-O Bf5 11. Bf4 9
1. e4 Nf

In [59]:
openings_group_counts = lessons_openings_data_frame[["first", "opening"]].value_counts()
openings_group_counts

first   opening                
e4-e5   Ruy Lopez                  44
        Italian Game               27
e4-c5   Sicilian Defense           24
d4-d5   Semi-Slav Defense          16
        Queen's Gambit Declined    14
d4-Nf6  King's Indian Defense      14
        Benoni Defense              6
        Grünfeld Defense            6
        Queen's Gambit Declined     6
d4-d5   Tarrasch Defense            5
e4-e5   King's Gambit Accepted      5
e4-e6   French Defense              4
e4-e5   Petrov's Defense            4
d4-Nf6  Catalan Opening             4
e4-e5   Four Knights Game           4
d4-d5   Queen's Gambit Accepted     3
e4-Nf6  Alekhine Defense            3
d4-Nf6  Bogo-Indian Defense         3
e4-c6   Caro-Kann Defense           3
e4-e5   Scotch Game                 2
d4-d5   Slav Defense                2
d4-Nf6  Benko Gambit Accepted       1
        Tarrasch Defense            1
e4-e5   Philidor Defense            1
d4-Nf6  Semi-Slav Defense           1
        Nimzo-Indi

In [60]:
from re import sub

def snake_case(s):
    return "_".join(
        sub("([A-Z][a-z]+)", r" \1", sub("([A-Z]+)", r" \1", s.replace("-", " "))).split()
    ).lower()

In [61]:
from chess_gif.gif_maker import GIFMaker
import pathlib
from tqdm.notebook import tqdm

for index in tqdm(lessons_openings_data_frame.index):
    pgn: str = lessons_openings_data_frame["pgn"][index]
    first: str = lessons_openings_data_frame["first"][index]
    opening: str = lessons_openings_data_frame["opening"][index]
    half_moves: int = lessons_openings_data_frame["half_moves"][index]
    main_line_score: int = lessons_openings_data_frame["main_line_score"][index]
    opening_folder = snake_case(opening).replace(":", "").replace(",", "").replace("'", "_")
    opening_variation: str = lessons_openings_data_frame["opening_variation"][index]
    file_name = snake_case(opening_variation).replace(":", "").replace(",", "").replace("'", "_")
    gif_maker = GIFMaker(delay=500, h_margin=0, v_margin=0)
    pgn_bytes = str.encode(pgn)
    pgn_folder = f"./lessons/pgn/{first}/{opening_folder}"
    pathlib.Path(pgn_folder).mkdir(parents=True, exist_ok=True)
    pgn_file = open(f"{pgn_folder}/{main_line_score}_{file_name}_{half_moves}.pgn", "w")
    print(f"{first}/{file_name}", pgn)
    pgn_file.write(pgn)
    pgn_file.close()
    gif_folder = f"./lessons/gif/{first}/{opening_folder}"
    pathlib.Path(gif_folder).mkdir(parents=True, exist_ok=True)
    gif_maker.make_gif_from_pgn_file(
        f"{pgn_folder}/{main_line_score}_{file_name}_{half_moves}.pgn", f"{gif_folder}/{main_line_score}_{file_name}_{half_moves}.gif"
    )

  0%|          | 0/206 [00:00<?, ?it/s]

c4-e6/english_opening_symmetrical_hedgehog_flexible_formation 1. c4 e6 2. Nf3 c5 3. Nc3 Nf6 4. g3 b6 5. Bg2 Bb7 6. O-O Be7 7. d4 cxd4 8. Qxd4 d6 9. Rd1 a6 10. b3 Nbd7
d4-Nf6/benko_gambit_accepted_king_walk_variation 1. d4 Nf6 2. c4 c5 3. d5 b5 4. cxb5 a6 5. bxa6 g6 6. Nc3 Bxa6 7. Nf3 d6 8. e4 Bxf1 9. Kxf1 Bg7 10. g3 O-O 11. Kg2
d4-Nf6/benoni_defense_classical_variation_czerniak_defense_tal_line 1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2
d4-Nf6/benoni_defense_classical_variation_czerniak_defense 1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2 Na6 11. f3
d4-Nf6/benoni_defense_classical_variation_full_line 1. d4 Nf6 2. c4 c5 3. d5 e6 4. Nc3 exd5 5. cxd5 d6 6. e4 g6 7. Nf3 Bg7 8. Be2 O-O 9. O-O a6 10. a4
d4-Nf6/benoni_defense_classical_variation_czerniak_defense 1. d4 Nf6 2. c4 e6 3. Nf3 c5 4. d5 exd5 5. cxd5 d6 6. Nc3 g6 7. e4 Bg7 8. Be2 O-O 9. O-O Re8 10. Nd2 Na6
d4-Nf6/benoni_d